In [1]:
import time
import sys
import os
# reload(sys)
# sys.setdefaultencoding('utf8')
import numpy as np
import pandas as pd
import datetime
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.utils.data as data_
import torch
import argparse
from torch.utils.data import IterableDataset
from datetime import datetime
import itertools
import io
from random import shuffle
import torch.nn.functional as F
import random
import math
import pickle
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import torch.nn.utils.rnn as rnn_utils
from torch.nn import Linear, ReLU, Sigmoid, Module, BCELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau,CosineAnnealingLR,StepLR
from torch.nn.utils import weight_norm
from math import sqrt
import torch.nn.functional as F
import hashlib

In [2]:
import pickle


with open('./data_bins/bin_edges_dict_dz_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_dz_numeric = pickle.load(handle)

    
with open('./data_bins/bin_edges_dict_inquery_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_inquery_numeric = pickle.load(handle)
print(bin_edges_dict_inquery_numeric)
   
    
with open('./data_bins/bin_edges_dict_creditos_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_creditos_numeric = pickle.load(handle)    
    
with open('./data_bins/bin_edges_dict_person_numeric_fusing_v5_8bin.pkl', 'rb') as handle:
    bin_edges_dict_person_numeric = pickle.load(handle)

{'inquiry_amt': Index([1000.0, 250000000.0], dtype='float64'), 'inquiry_date_ago': Index([58.0, 119.0, 190.0, 272.0, 365.0, 468.0, 588.0, 730.0], dtype='float64')}


In [3]:
class Tokenizer:
    """A simple tokenizer to convert between tokens and indices."""
    
    def __init__(self, token_to_index=None, index_to_token=None):
        """
        Initializes the tokenizer.

        Args:
            token_to_index (dict, optional): A dictionary mapping tokens to their indices.
            index_to_token (dict, optional): A dictionary mapping indices back to tokens.
        """
        self.token_to_index = token_to_index or {}
        self.index_to_token = index_to_token or {}
        self.unk_token = "<unk>"  # The token representing unknown words.
        self.unk_index = 0  # Default index for unknown tokens (`<unk>`).

    def fit(self, data):
        """
        Fits the tokenizer on the data to build the mapping dictionaries.

        Args:
            data (list): The dataset containing tokens to build the mappings from.
        """
        unique_tokens = sorted(set(data))  # Ensure consistent order.
        self.token_to_index = {token: index for index, token in enumerate(unique_tokens, start=1)}
        self.index_to_token = {index: token for token, index in self.token_to_index.items()}
        self.unk_index = 0  # Reset the index for `<unk>` in case it changes.

    def transform(self, data):
        """
        Transforms the data into indices using the learned token-to-index map.

        Args:
            data (list): The data to be transformed into indices.

        Returns:
            list: A list of indices corresponding to the tokens in `data`.
        """
        return [self.token_to_index.get(item, self.unk_index) for item in data]

    def reverse_transform(self, data):
        """
        Converts indices back into tokens using the index-to-token map.

        Args:
            data (list): The indices to be converted back into tokens.

        Returns:
            list: A list of tokens corresponding to the indices in `data`.
        """
        return [self.index_to_token.get(index, self.unk_token) for index in data]


# Define a list of inquiry types.
inquiry_type_list = [
    'F', 'TC', 'NC', 'M', 'PP', 'R', 'Q', 'LC', 'CA', 'OT', 'AM', 'PG', 'E', 'PN', 'AE', 'P', 'PM', 'AR', 
    'NG', 'A', 'FI', 'L', 'null', 'TG', 'FT', 'LR', 'CO', 'CF', 'EQ', 'VR', 'ED', 'AV', 'MC', 'BL', 'SH', 'HE'
]

# Create a tokenizer for inquiry types.
inquiry_type_tokenizer = Tokenizer()
inquiry_type_tokenizer.fit(inquiry_type_list)

# Define a list of income categories.
month_income_list = [
    '5,000-10,000MXN', '10,000-20,000MXN', '3,000-5,000MXN', '20,000-50,000MXN', '1,000-3,000MXN', 'below 1,000MXN', 'above 50,000MXN'
]

# Create a tokenizer for monthly income categories.
month_income_tokenizer = Tokenizer()
month_income_tokenizer.fit(month_income_list)



def map_values_to_bins(values_str, bin_edges):
    values = np.array(list(map(float, values_str.split(','))))  # 将字符串值转换成浮点数列表
    binned_values =  np.digitize(values, bins=bin_edges, right=True).tolist()
    binned_values = [int(x) if not (x == float('inf') or x == float('-inf')) else 0 for x in binned_values]

    return binned_values


def map_values_to_bins_single(values_str, bin_edges):
    values = np.array(list(map(float, values_str.split(','))))  # 将字符串值转换成浮点数列表
    binned_values =  np.digitize(values, bins=bin_edges, right=True).tolist()[0]
#     print(binned_values)
    return binned_values


class MyData(IterableDataset):
    def __init__(self, file_paths, is_train=True, augmentation_prob=0.8,shuffle_files=True):
        if shuffle_files:
            shuffle(file_paths)
        self.file_paths = file_paths
        self.is_train = is_train
        self.augmentation_prob = augmentation_prob
        
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        world_size = 1
        rank = 0

        worker_id = 0
        mod = world_size
        shift = rank

        if worker_info:
            num_workers = worker_info.num_workers
            worker_id = worker_info.id
            mod *= num_workers
            shift = rank * num_workers + worker_id


        line_iter = itertools.chain(*[io.open(f, encoding="utf-8") for f in self.file_paths])
        for i, line in enumerate(line_iter):
            if (i + shift) % mod == 0:
                return_line = self.process(line)
                if return_line:
                    yield return_line


    def process(self, line):
        text_data = line
        dz_categorica_feature, dz_numeric_feature, person_feature , label, \
        len_dz, cfrnid, bv1_score,date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos = process_one_sample(text_data)
        dz_categorica_feature = torch.tensor(dz_categorica_feature)
        dz_numeric_feature = torch.tensor(dz_numeric_feature)
        person_feature = torch.tensor(person_feature)
        label = torch.tensor(label)
        if cfrnid != 360287970202376449:
            return False
        cfrnid = torch.tensor(cfrnid)
        bv1_score = torch.tensor(bv1_score)
        date_time_credit = torch.tensor(date_time_credit)
        
        inquery_feature = torch.tensor(inquery_feature)
        creditos_feature = torch.tensor(creditos_feature)
        
        tuple_ = (dz_categorica_feature, dz_numeric_feature, person_feature,label, len_dz, cfrnid, bv1_score,date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos)
        return tuple_
    
    

def process_2d_list(lst, k, pad_token=120):
    result = []
    for sub_lst in lst:
        if len(sub_lst) < k:
            sub_lst += [pad_token] * (k - len(sub_lst))
            result.append(sub_lst)
        else:
            result.append(sub_lst[-k:])
    return result


In [48]:
def process_one_sample(features,max_length=120):
    feature1, feature2, len1, label1, label2, label3, feature3, feature4, feature5, feature6, feature7, feature8 = features.split('\t')
    feature2 = feature2.strip('\n')  
    label = [int(label1),int(label2),int(label3)]  
    feature1 = float(feature1)    
    cls_token = 121 # Define class sequence feature token to represent one sentence
    
    # Example
    feature1 = month_income_tokenizer.transform([feature1])[0]
    feature2 = inquiry_type_tokenizer.transform([feature2])[0]
    person_categorica_feature = [feature1]
    person_numeric_feature = [feature2]
    person_numeric_feature_discretized = [map_values_to_bins_single(data_str, bin_edges_dict_person_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_person_numeric.keys()), person_numeric_feature)]
    person_feature = person_categorica_feature  + person_numeric_feature_discretized
    dz_categorica_feature = [feature3]
    dz_categorica_feature = [fe.split(',') for fe in dz_categorica_feature]
    dz_numeric_feature = [feature4]
    dz_numeric_feature = [map_values_to_bins(data_str, bin_edges_dict_dz_numeric[feature_name]) for feature_name, data_str in zip(list(bin_edges_dict_dz_numeric.keys()), dz_numeric_feature)]
 
    max_length = 25
    len_dz= len(dz_categorica_feature[0])
    len_dz = len_dz if len_dz <=max_length else max_length
    dz_categorica_feature = [[int(value) for value in row] for row in dz_categorica_feature]
    dz_numeric_feature = [[float(value) if value != '' else 0.0 for value in row] for row in dz_numeric_feature]
    dz_categorica_feature = process_2d_list(dz_categorica_feature,max_length)
    # Adjust features to max_length and prepend class token
    dz_categorica_feature = [[cls_token] + sublist for sublist in dz_categorica_feature]
    dz_numeric_feature = process_2d_list(dz_numeric_feature,max_length)
    dz_numeric_feature = [[cls_token] + sublist for sublist in dz_numeric_feature]
    
    print('dz_categorica_feature summary include ', len(dz_categorica_feature), 'sentences',  ' and dz_categorica_feature one sentence like : ', dz_categorica_feature[1])
    print('dz_numeric_feature summary include : ', len(dz_numeric_feature), 'sentences', ' and dz_numeric_feature one sentence like : ', dz_numeric_feature[3])
    print('inquery_feature summary include : ', len(inquery_feature), 'sentences', ' and inquery_feature one sentence like : ', inquery_feature[4])
    print('creditos_feature summary include : ', len(creditos_feature), 'sentences', ' and creditos_feature one sentence like : ', creditos_feature[2])

    return dz_categorica_feature, dz_numeric_feature, person_feature, label, len_dz
   
    
    

In [49]:
import json
import torch  # Assume that PyTorch is used for model evaluation

def load_data_from_json(file_path):
    """
    Load and return data from a JSON file.

    Parameters:
    - file_path (str): The path to the JSON file.

    Returns:
    - dict: The data loaded from the JSON file as a dictionary.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def evaluate(input_data):
    """
    Evaluate the model with features extracted from the input data.

    Parameters:
    - input_data (dict): The dictionary containing features and data to evaluate.

    Raises:
    - Exception: Throws an exception if necessary features are missing, or if there's an error during preprocessing or model prediction.

    Notes:
    - Assume process_one_sample and other functions are defined elsewhere and handle specific data preparation and model evaluation tasks.
    """
    try:
        data_input = []
        features_list = columns_num  # Assume columns_num is defined elsewhere and contains the list of required features
        for feature in features_list:
            data_input.append(input_data[feature])
    except Exception as e:
        raise Exception("Incomplete features for model input, please check your configuration!") from e

    try:
        # Example function call - replace with actual preprocessing and evaluation
        dz_categorica_feature, dz_numeric_feature, person_feature, label, len_dz, cfrnid, bv1_score, date_time_credit, inquery_feature, creditos_feature, len_inquery, len_creditos = process_one_sample(data_input)
        
    except Exception as e:
        raise Exception("Error during feature preprocessing!") from e
    
    try:
        val_preds = []
        val_score = []
        # Model evaluation logic goes here
        with torch.no_grad():
            pass  # Replace with actual evaluation logic
        
    except Exception as e:
        raise Exception("Anomaly during model prediction process!") from e
        
file_path = 'path_to_your_file.json'  # Specify the path to your JSON file
file_path = './data.json'
# Load data from JSON file
data_dict = load_data_from_json(file_path)

# Evaluate the model with loaded data
evaluate(data_dict)

dz_categorica_feature summary include  4 sentences  and dz_categorica_feature one sentence like :  [121, 0, 0, 1, 1, 1, 1, 1, 1, 1, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120]
dz_numeric_feature summary include :  237 sentences  and dz_numeric_feature one sentence like :  [121, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120]
inquery_feature summary include :  5 sentences  and inquery_feature one sentence like :  [121, 7, 7, 7, 6, 6, 5, 4, 4, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 12